<center><h1>Semantic Search</h1></center>
<center> - </center>
<center> Exploring Embedding techniques and Similarity Measures </center>

The task is to develop a semantic search engine. We have a corpus of texts. Since there is a large amount of documents, we want to run small search queries so that is returns texts that are semantically close to the query.

The process can be illustrated the following way :

<img src="images/process.png">

# Imports

In [8]:
### General ###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt; plt.rcdefaults()
import re

from sklearn.model_selection import train_test_split

### Text processing ###
from nltk import wordpunct_tokenize, WordNetLemmatizer, sent_tokenize, pos_tag
from nltk.corpus import stopwords as sw, wordnet as wn
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import string 
import spacy
from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer

### Pre-trained model ###
import tensorflow as tf
import tensorflow_hub as hub

### Live Search Engine ###
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

In [9]:
EN = spacy.load('en')

# The data

### Elon Musk's Tweets

To replicate the kind of messages we would observe in a message app, I have chosen to explore Elon Musk's tweets data set : https://data.world/adamhelsinger/elon-musk-tweets-until-4-6-17

In [185]:
tweets = pd.read_csv('tweets.csv')['text']
tweets.head(10)

0    b'And so the robots spared humanity ... https:...
1    b"@ForIn2020 @waltmossberg @mims @defcon_5 Exa...
2        b'@waltmossberg @mims @defcon_5 Et tu, Walt?'
3                  b'Stormy weather in Shortville ...'
4    b"@DaveLeeBBC @verge Coal is dying due to nat ...
5    b"@Lexxxzis It's just a helicopter in helicopt...
6                            b"@verge It won't matter"
7                        b'@SuperCoolCube Pretty good'
8    b"Why did we waste so much time developing sil...
9    b'Technology breakthrough: turns out chemtrail...
Name: text, dtype: object

In [186]:
tweets.shape

(2819,)

### South Park Series Data

https://github.com/BobAdamsEE/SouthParkData

In [72]:
south = pd.read_csv('All-seasons.csv')['Line'][:1000]
south.head(10)

0           You guys, you guys! Chef is going away. \n
1                          Going away? For how long?\n
2                                           Forever.\n
3                                    I'm sorry boys.\n
4    Chef said he's been bored, so he joining a gro...
5                                               Wow!\n
6    Chef?? What kind of questions do you think adv...
7       What's the meaning of life? Why are we here?\n
8             I hope you're making the right choice.\n
9    I'm gonna miss him.  I'm gonna miss Chef and I...
Name: Line, dtype: object

In [73]:
south.shape

(1000,)

# I . Pre-processing 

The first step our the semantic similarity search is to pre-process the text before apply any similarity metric or embedding on it. 

There are however several technical challenges :
- Removal of stop words should be discussed, as it might be tricky if the search query contains several stop words
- Users could be looking for specific words within an URL, so we should not remove the URLS
- Split the text by sentences

In [74]:
def preprocess(document, max_features = 150, max_sentence_len = 300):
    """
    Returns a normalized, lemmatized list of tokens from a document by
    applying segmentation (breaking into sentences), then word/punctuation
    tokenization, and finally part of speech tagging. It uses the part of
    speech tags to look up the lemma in WordNet, and returns the lowercase
    version of all the words, removing stopwords and punctuation.
    """
    
    i = 0
    
    def lemmatize(token, tag):
        """
        Converts the tag to a WordNet POS tag, then uses that
        tag to perform an accurate WordNet lemmatization.
        """
        tag = {
        'N': wn.NOUN,
        'V': wn.VERB,
        'R': wn.ADV,
        'J': wn.ADJ
        }.get(tag[0], wn.NOUN)

        return WordNetLemmatizer().lemmatize(token, tag)

    def vectorize(doc, max_features, max_sentence_len):
        """
        Converts a document into a sequence of indices of length max_sentence_len retaining only max_features unique words
        """
        tokenizer = Tokenizer(num_words=max_features)
        tokenizer.fit_on_texts(doc)
        doc = tokenizer.texts_to_sequences(doc)
        doc_pad = pad_sequences(doc, padding = 'pre', truncating = 'pre', maxlen = max_sentence_len)
        return np.squeeze(doc_pad), tokenizer.word_index

    lemmatized_tokens = []

    # Clean the text using a few regular expressions
    document = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", document)
    document = re.sub(r"what's", "what is ", document)
    document = re.sub(r"\'", " ", document)
    document = re.sub(r"@", " ", document)
    document = re.sub(r"\'ve", " have ", document)
    document = re.sub(r"can't", "cannot ", document)
    document = re.sub(r"n't", " not ", document)
    document = re.sub(r"i'm", "i am ", document)
    document = re.sub(r"\'re", " are ", document)
    document = re.sub(r"\'d", " would ", document)
    document = re.sub(r"\'ll", " will ", document)
    document = re.sub(r"(\d+)(k)", r"\g<1>000", document)
    document = document.replace("\n", " ")
    
    cleaned_document = []
    
    # Break the document into sentences
    for sent in sent_tokenize(document):
        lemmatized_tokens = []

        # Break the sentence into part of speech tagged tokens
        for token, tag in pos_tag(wordpunct_tokenize(sent)):

            # Apply preprocessing to the tokens
            token = token.lower()
            token = token.strip()
            token = token.strip('_')
            token = token.strip('*')

            # If punctuation ignore token and continue
            if all(char in set(string.punctuation) for char in token): #token in set(sw.words('english')) or 
                continue

            # Lemmatize the token
            lemma = lemmatize(token, tag)
            lemmatized_tokens.append(lemma)

        cleaned_document.append(' '.join(lemmatized_tokens))
    
    
    #vectorized_document, word_index = vectorize(cleaned_document, max_features, max_sentence_len)
    #return vectorized_document, word_index
    
    return cleaned_document

In [76]:
south = preprocess(str(list(south)))

In [77]:
south

['you guy you guy',
 'chef be go away',
 'n go away for how long n forever',
 'n i m sorry boy',
 'n chef say he s be bore so he join a group call the super adventure club',
 'n wow',
 'n chef what kind of question do you think adventure around the world be gonna answer',
 'n what s the meaning of life why be we here n i hope you re make the right choice',
 'n i m gonna miss him',
 'i m gonna miss chef and i and i don t know how to tell him',
 'n dude how be we gonna go on chef be our fuh f ffriend',
 'n and we will all miss you chef but we know you must do what your heart tell you n bye bye',
 'n good bye',
 'n so long',
 'n so long chef',
 'n good bye chef',
 'n good bye chef',
 'have a great time with the super adventure club',
 'n good bye',
 'n draw two card fatass',
 'n reverse to you jew',
 'n i ll get it',
 'n hello there child',
 'n he s back',
 'n yeah',
 'n all right',
 'n chef',
 'i can t believe you re back',
 'n well it s true',
 'n but be you back for good n that s right

# II . Cosine similarity on TF-IDF

The TF-IDF weight of a word w in a document d belonging to a corpus is the ratio of the number of times w occurs in the document d to the number of documents in which w occurs at least ones. This weight reflects the specificity and the importance of a word within a document.

With cosine similarity, we need to convert sentences into vectors. One way to do that is to use :

Bag of words with either TF (term frequency)
TF-IDF (term frequency- inverse document frequency).
Bag of Words techniques are simple one-hot encoding of the data, on which we can decide to apply a principal component analysis (PCA).

TF is good for text similarity in general, but TF-IDF is good for search query relevance. Using cosine distance over TF-IDF vectors is, therefore, suitable for our search query relevance case.



<img src="cosine.png">

**Advantage** : Easy to implement, fast to run

**Disdvantage** : State of the art embedding produce better results, do not take order or words into account


Extensions : Smooth Inverse Frequency

# III. Knowledge-based measures using WordNet

# IV . Embedding

There are 2 ways to embed a search query :

- embed the words using Word2Vec for example (and compute the average embedding of the sentence for example)
- embed the whole sentence

The first word embedding technique as mentioned above is the simple one-hot encoding, which is called Bag-Of-Words, but this typically creates sparse matrices that are just too large, and on which we apply PCA or T-SNE. We however want to capture some of the connections between the words in our embedding.

## 1. Word Embedding

Word embedding techniques like Word2Vec are interesting since once we've created an embedding, we can locate this vector on a vector space. Word2vec burst onto the scene in 2013 and entirely changed the NLP landscape, demonstrating that deep learning methods could yield incredible results when applied to NLP tasks such as translation, classification, and semantic parsing. Word2vec, and similar approaches such as GloVe, showed that vector representations of words could be used to discover semantic meaning without requiring domain specific knowledge.

Skip-gram and CBOW enabled word2vec models to be trained without requiring any labelling of data. The key to both approaches is to use the context of the words in a sentence to train the network. The diagram shows the input and output for each method.

For skip-gram, the input of the model is a middle word in a sequence. The network then tries to predict the surrounding words. For CBOW, the input of the model is the surrounding words and the network tries to predict the middle word.

<img src = "images/word.png">

Fortunately, we don’t have t train our own CBOW and Skip Gram models in order to obtain word embeddings : it is possible to load pretrained embeddings and use them into our models (https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit). In order to load the Google Word2Vec embedding dictionnary and select the relevant items corresponding to a preprocessed corpus, it is possible to use a code similar to the following.

In [ ]:
def load_google_vec(): 
    return KeyedVectors.load_word2vec_format('GoogleWord2Vec300.bin', binary=True)

def prepare_embedding(max_features, max_sentence_len, embed_dim, corpus):

    word2vec = load_google_vec()
    corpus_preprocessed, train_word_index = preprocess(X, max_features, max_sentence_len)
    train_embedding_weights = np.zeros((len(train_word_index)+1, embed_dim))

    for word,index in train_word_index.items():
        train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(embed_dim)
    
    word_vector_dict = dict(zip(pd.Series(list(train_word_index.keys())),pd.Series(list(train_word_index.keys())).apply(lambda x: train_embedding_weights[train_word_index[x]])))
    
    return train_embedding_weights, train_word_index, word_vector_dict

***Advantage :***

Pre-trained model, should not perform too bad

***Disadvantage :***

- They ignore word ordering
- It's difficult to capture the semantic meaning of a sentence
- Sentence length becomes problematic. If we just use word embeddings, it can be difficult to discover if sentences of different lengths are similar
- They introduce extra complexity


Limitations of word embedding can be overcome using :

the average embedding vector of all words of a sentence
more advanced method which attempts to add a weighting function to word embeddings which down-weights common words. This latter approach is known as Smooth Inverse Frequency (SIF).

## 2. Rolling Window Embedding

As mentioned above, one of the issues is to encode the whole sentence, or a given sequence of words. A first way to approach this is to consider a fixed length of words, embed it (using onversation with 250 words. Take a window of 10 words, encode them, gives a vector, move to encode the 10 next, with padding of 4, until the end. We end up with a bunch a vectors, and then do the search. This enhances the accuracy of semantic similarity because the scope is smaller. 

<img src = "images/rw.png">

This architecture also allows to highlight the match.

## 3. Sentence Embedding

### a. Test the embedding

In [78]:
# Download the USE module
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" 
embed = hub.Module(module_url)

In [79]:
# Generate the embedding and print out some descriptive data
def generate_embedding(messages):
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        message_embeddings = session.run(embed(messages))
    for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
        print("Message: {}".format(messages[i]))
        print("Embedding size: {}".format(len(message_embedding)))
        message_embedding_snippet = ", ".join(
            (str(x) for x in message_embedding[:3]))
        print("Embedding: [{}, ...]\n".format(message_embedding_snippet))
    return(message_embeddings[0])

Let's try to generate a first embedding :

In [80]:
emb = generate_embedding(["How can I reset my password"])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0329 00:13:06.389371 4579259840 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


Message: How can I reset my password
Embedding size: 512
Embedding: [0.025553924962878227, -0.034720830619335175, 0.0020717347506433725, ...]



### b. Embed the whole file

In [81]:
session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())

In [82]:
sts_input1 = tf.placeholder(tf.string, shape=(None))

# For evaluation we use exactly normalized rather than
# approximately normalized.
sts_encode1 = tf.nn.l2_normalize(embed(sts_input1), axis=1)

def get_embeds(session, text):
    """Returns the similarity scores"""
    embed = session.run(
        [sts_encode1],
        feed_dict={
            sts_input1: text
        })
    return(embed[0][0])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0329 00:13:13.072659 4579259840 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [84]:
%time
embed_south = []

for message in range(len(south)) :
    embed_south.append(get_embeds(session, [south[message]]))

CPU times: user 8 µs, sys: 2 µs, total: 10 µs
Wall time: 19.1 µs


In [85]:
south_df = pd.DataFrame(south)
south_df['embed'] = embed_south
south_df.head()

,0,embed
0,you guy you guy,"[-0.043390382, -0.027857497, 0.03543695, -0.01..."
1,chef be go away,"[-0.04750077, -0.033675905, 0.07155576, 0.0198..."
2,n go away for how long n forever,"[0.0413956, 0.00991075, 0.051509243, -0.021212..."
3,n i m sorry boy,"[0.042864855, -0.0409749, -0.007703394, 0.0196..."
4,n chef say he s be bore so he join a group cal...,"[-0.09336378, 0.07813314, 0.066432685, -0.0846..."


In [86]:
south_df['embed'] = south_df['embed'].apply(lambda x : x.astype('float'))

In [87]:
south_df.head()

,0,embed
0,you guy you guy,"[-0.04339038208127022, -0.027857497334480286, ..."
1,chef be go away,"[-0.047500770539045334, -0.0336759053170681, 0..."
2,n go away for how long n forever,"[0.04139560088515282, 0.009910750202834606, 0...."
3,n i m sorry boy,"[0.042864855378866196, -0.040974900126457214, ..."
4,n chef say he s be bore so he join a group cal...,"[-0.09336377680301666, 0.07813314348459244, 0...."


### c. Define semantic similarity metric

In [88]:
sts_input1 = tf.placeholder(tf.string, shape=(None))
sts_encode2 = tf.placeholder(tf.float32)

# For evaluation we use exactly normalized rather than
# approximately normalized.
sts_encode1 = tf.nn.l2_normalize(embed(sts_input1), axis=1)

cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
clip_cosine_similarities = tf.clip_by_value(cosine_similarities, 0.0, 1.0)
sim_scores = 1.0 - tf.divide(tf.acos(clip_cosine_similarities), 3.14)

def get_scores(session, text_a, text_b):
    """Returns the similarity scores"""
    scores= session.run(
        [sim_scores],
        feed_dict={
            sts_input1: text_a,
            sts_encode2: text_b
        })
    return(scores)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0329 00:15:48.271921 4579259840 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [89]:
def get_results(sessions, sentence, num):
    examples = [e for e in south_df['embed']]
    scores = get_scores(session, [sentence], examples)
    south_df['cosine'] = scores[0].tolist()
    return(south_df.sort_values('cosine', ascending=False).head(n=num))

In [90]:
def print_res(test, num=20):
    res = get_results(session, test, num).round(4)
    res = (res.set_index('cosine'))
    print('{}\n'.format(test))
    print('\x1b[31mScore{:<1} \x1b[0m: \x1b[34m Matching sentence\x1b[0m'.format(''))
    for i in res.iterrows():
        print('\x1b[31m{:<6} \x1b[0m: \x1b[0m \x1b[34m{}\x1b[0m'.format(i[0], i[1][0]))

### d. Live search prediction

In [96]:
@register_cell_magic
def search(line, cell):
    return print_res(cell)

In [106]:
%%search
"I'm unemployed"

"I'm unemployed"

Score  :  Matching sentence
0.7328 :  i need to shape up and find a new job quick
0.7085 :  the rent s due at the end of the month and i don t have any money
0.6875 :  and if you all don t want to work tomorrow you can just find other job
0.649  :  n i shouldn t even be in the office still
0.6432 :  i should become a writer
0.6397 :  if i just write a book about my life i can get it publish and then make plenty of money to pay rent
0.6378 :  do thing for my own selfish reason
0.637  :  n hey brian this be like the time i get a job a a carrot cake
0.6357 :  n be it work be it work n yes
0.6346 :  quiet student quiet
0.6345 :  now they know all they have to do be refuse to work and they can get whatever they want
0.6319 :  i make them leave
0.6319 :  i run this network not you guy
0.6311 :  n i ll do everything i can
0.6298 :  it s suppose to be half day friday
0.6275 :  i ve learn how to make the family guy write staff stop working
0.6258 :  they just stop work
0.6246 

# V . Sources and Resources

- Different Techniques : http://www.lumenai.fr/blog/quick-review-on-text-clustering-and-text-similarity-approaches
- LSI (1) : https://nlp.stanford.edu/IR-book/html/htmledition/latent-semantic-indexing-1.html
- LSI (2) : https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python
- WordNet : https://www.coursera.org/lecture/python-text-mining/semantic-text-similarity-DpNWl
- Universal Sentence Encoding (1) : https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/semantic_similarity_with_tf_hub_universal_encoder.ipynb#scrollTo=zwty8Z6mAkdV
- Universal Sentence Encoding (2) : https://arxiv.org/pdf/1804.07754.pdf
- Semantic Similarity in Python : https://towardsdatascience.com/semantic-code-search-3cd6d244a39c
- Github repository with useful content: https://github.com/hamelsmu/code_search/tree/master/notebooks
- Automate Customer Support (1) : https://blog.floydhub.com/automate-customer-support-part-one/
- Automate Customer Support (2) : https://blog.floydhub.com/automate-customer-support-part-two/
- https://github.com/choran/sentence_embeddings